## Исследование надёжности заёмщиков

На основе статистики о платёжеспособности клиентов необходимо выяснить влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.

### Шаг 1. Загрузка данных 

In [1]:
# Импорт библиотек
import pandas as pd
from IPython.display import display
import warnings
warnings.simplefilter("ignore")

Прочитаем файл datasets/data.csv и сохраним его в переменной data.

In [2]:
data = pd.read_csv('/datasets/data.csv')

Получение первых строк таблицы.

In [3]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Вывод общей информации о данных в таблице data.

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод 
Рассмотрим полученную информацию подробнее.

Всегов таблице 21525 строк и 12 столбцов. Тип данных у двух столбцов - float, у пяти - int64, у пяти - object.

Подробно разберём, какие в data столбцы и какую информацию они содержат:

children — количество детей в семье;
days_employed — общий трудовой стаж в днях;
dob_years — возраст клиента в годах;
education — уровень образования клиента;
education_id — идентификатор уровня образования;
family_status — семейное положение;
family_status_id — идентификатор семейного положения;
gender — пол клиента;
income_type — тип занятости;
debt — имел ли задолженность по возврату кредитов;
total_income — ежемесячный доход;
purpose — цель получения кредита.

Количество значений в столбцах различается. Это говорит о том, что в данных days_employed (общий трудовой стаж в днях) и total_income (ежемесячный доход) есть пропущенные значения.

Каждая строка таблицы содержит информацию о клиенте банка и его платежеспособности. Две проблемы, которые нужно решать: пропуски и некачественные значения в столбцах. Для проверки рабочих гипотез особенно ценны столбцы children, family_status. Данные из столбца debt позволят узнать был ли погашен кредит в срок.

### Шаг 2. Предобработка данных

Исключим пропуски, а также проверим данные на наличие дубликатов.

### Обработка пропусков

Проверим данные на наличие пропусков вызовом набора методов для суммирования пропущенных значений.

In [5]:
print(data.isnull().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


Пустые значения свидетельствуют, что для некоторых клиентов доступна не вся информация. В исходных данных отсутствуют 2174 записи о трудовом стаже и 2174 записи о ежемесячном доходе. Причины могут быть разные, необходимо разобраться и выявить причину:сознательные это пропуски или это проблемы с записью данных. Так как разброс в данных может быть очень большой, то чтобы сопоставить их заменим пропуски не просто медианными значениями, а медианными значениями по группам (по категориям). Пропуски в days_employed заполним медианой по группировке по типу занятости и возрастной категории. Пустые значения о ежемесячном доходе заполняим медианными значениями по каждому типу занятости.

Заполнение пропущенных значений ежемесячного дохода медианными значениями по каждому типу занятости. 

In [6]:
#Типы занятости многократно повторяются в данных. Создадим словарь типов занятости.
#Подсчитаем медианное значение для каждого типа занятости и сохраним в переменную data_total_income_median

data_total_income_median = data[['income_type','total_income']].groupby('income_type')['total_income'].median()
print(data_total_income_median)


income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64


In [7]:
#Создаем цикл по уникальным значениям типа занятости и заменяем пропуски медианными значениями
for income_type in data['income_type'].unique():
    income_type_mediana = data[data['income_type'] == income_type]['total_income'].median()
    data.loc[(data['total_income'].isna()) & (data['income_type'] == income_type),'total_income'] = income_type_mediana

data[data['days_employed']==0]['total_income'] #-проверка, что пропуски заполнились медианой

Series([], Name: total_income, dtype: float64)

Чтобы провести анализ по возрастным группам, необходимо предварительно объединенить данных в категории. 
Распределим клиентов так: 
-клиенты младше 30 лет попадают в категорию 'молодежь';
-клиенты от 30 до 64 лет — категория 'взрослые';
-клиенты старше 65 лет принадлежат к категории 'пенсионеры'.
Создадим функцию для такой категоризации. 

In [8]:
def age_groups(age):
    if age <= 30:
        return 'молодежь'
    if age <= 64:
        return 'взрослые'
    return 'пенсионеры'

#На вход функции попадает возраст, а возвращает она категорию клиента.
#Возвращает возврастную группу по значению возраста age, используя правила: 
#- 'молодежь' при значении age <= 30 лет
#- 'взрослые' при значениии age более 30 и менее 64, включая 64
#- 'пенсионеры' во всех остальных случаях

data['age_group'] = data['dob_years'].apply(age_groups)
#Создадим отдельный столбец с возрастными категориями 'age_group', и в его ячейках запишем значения, возвращаемые функцией.
#Для этого вызовем метод apply() для столбца 'dob_years', так как в нём содержатся данные, которые функция примет на вход. 
#Аргументом метода станет сама функция age_groups.

print(data[['dob_years','age_group']].head(15))
   

    dob_years   age_group
0          42    взрослые
1          36    взрослые
2          33    взрослые
3          32    взрослые
4          53    взрослые
5          27    молодежь
6          43    взрослые
7          50    взрослые
8          35    взрослые
9          41    взрослые
10         36    взрослые
11         40    взрослые
12         65  пенсионеры
13         54    взрослые
14         56    взрослые


Заметим, что в исходных данных в столбце days_employed встречаются отрицательные значения, но этого не может быть, так как стаж - величина неотрицательная. Необходимо сообщить об этом заказчику и выяснить причину данной неточности. Возможно, это ошибка в выгрузке, поэтому заменим отрицательные значения на положительные.

In [9]:
# Создадим функцию, которая отрицательные значения умножает на (-1), чтобы преобразовать число в положительное.
def poloz(value):
    if value<0: return value*(-1)
    else: return value
    
# Применим функцию poloz к столбцу days_employed
data['days_employed'] = data['days_employed'].apply(poloz)

Пропуски в days_employed заполним медианой по группировке по типу занятости и возрастной категории.

In [10]:
#Создаем цикл по по возрастным категориям внутри цикла по уникальным значениям типа занятости
#и заменяем пропуски медианными значениями
for income_type in data['income_type'].unique():
    for age in data['age_group'].unique():
        days_employed_mediana = data[(data['income_type'] == income_type)&(data['age_group'] == age)]['days_employed'].median()
        data.loc[(data['days_employed'].isna()) & (data['income_type'] == income_type)&(data['age_group'] == age),'days_employed'] = days_employed_mediana


Найдем строки, в которых остались пустые значения.

In [11]:
data[data['days_employed'].isnull()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
5936,0,NaN,58,высшее,0,женат / замужем,0,M,предприниматель,0,499163.144947,покупка жилой недвижимости,взрослые


In [12]:
data[data['income_type']=='предприниматель']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
5936,0,NaN,58,высшее,0,женат / замужем,0,M,предприниматель,0,499163.144947,покупка жилой недвижимости,взрослые
18697,0,520.848083,27,высшее,0,гражданский брак,1,F,предприниматель,0,499163.144947,на проведение свадьбы,молодежь


Заменим пропуск в полученной строке на 0, так как для нее нельзя определить медианное значение. В данных всего одна строка с пропущенным общим трудовым стажем в днях у взрослого предпринимателя.

In [13]:
data['days_employed'] = data['days_employed'].fillna(0)

Проверка, что таблица больше не содержит пропусков 

In [14]:
print(data.isnull().sum())


children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
age_group           0
dtype: int64


### Вывод
В исходных данных отсутствуют 2174 записи о трудовом стаже и 2174 записи о ежемесячном доходе. Это количественные переменные. Пропуски в таких переменных заполняют характерными значениями. Заменили пропуски в данных о стаже (days_employed) на медианные значения по группировке по типу занятости и возрастной категории. Заметим, что в данных были отрицательные значения стажа, их заменили на соответствующие положительные значения. Пропуски о ежемесячном доходе заполнили медианными значениями, взятыми по каждому типу занятости.

### Замена типа данных

Заменим вещественный тип данных на целочисленный

In [15]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')

Проверим результат выводом информации об исходной таблице.

In [16]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
age_group           21525 non-null object
dtypes: int64(7), object(6)
memory usage: 2.1+ MB
None


### Вывод
У столбцов days_employed и total_income тип данных float64, это количественные переменные. Количество дней трудового стажа - это целочисленный показатель, поэтому можем заменить тип данных на int64. Средний ежемесячный доход также может быть округлен до целочисленного значения, поэтому преобразовали и его.




### Обработка дубликатов

Необходимо установить наличие дубликатов. Если найдутся, удаляем, и проверяем, все ли удалились.

In [17]:
data.duplicated().sum()
#получение суммарного количества дубликатов в таблице data

54

In [18]:
data = data.drop_duplicates().reset_index(drop=True)
# удаление всех дубликатов из таблицы data специальным методом

In [19]:
data.duplicated().sum()
# проверка на отсутствие

0

Дубликаты могли появиться вследствие сбоя в записи данных. Стоит обратить внимание и разобраться с причинами появления такого «информационного мусора».

In [20]:
#Выведем статистику по количеству детей 'children' методом value_counts().
print(data['children'].value_counts()) 

 0     14107
 1      4809
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


Количество детей не может быть отрицательным, поэтому удалим строки со значением '-1'.
Строки с количеством детей 20 оставим, так как факт наличия детей необходим нам для анализа.

In [21]:
data = data[data['children'] >= 0] #отбираем ненулевые значения количества детей
print(data['children'].value_counts()) # проверка

0     14107
1      4809
2      2052
3       330
20       76
4        41
5         9
Name: children, dtype: int64


In [22]:
#Проверка встречающихся значений в столбце 'days_employed'
print(data['days_employed'].value_counts()) 

1820      830
1782      407
364944    321
1049      248
3110      121
         ... 
353413      1
378001      1
349331      1
332955      1
0           1
Name: days_employed, Length: 9081, dtype: int64


Общий трудовой стаж в днях - неотрицательное значение. В исходных данных встречаются отрицательные значения, что неестественно. Необходимо уточнить входные данные у заказчика.

In [23]:
# Выведем статистику по возрастам 'dob_years' методом value_counts().
print(data['dob_years'].value_counts())

35    615
40    605
41    604
34    598
38    595
42    594
33    579
39    572
31    557
36    554
29    544
44    543
30    537
48    536
37    534
43    511
50    511
49    508
32    507
28    501
45    497
27    492
56    484
52    484
47    477
54    474
46    470
53    458
58    456
57    454
51    447
55    442
59    442
26    407
60    374
25    357
61    353
62    349
63    268
24    264
64    261
23    252
65    194
22    183
66    182
67    167
21    111
0     101
68     99
69     84
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64


Есть 101 нулевое значение, этого не может быть, так как кредиты выдаются только совершеннолетним. Удалять эти строки не будем, так как эти данные не учитываются в нашем дальнейшем анализе.

In [24]:
# Проверяем значения в столбце 'education'.
print(data['education'].value_counts())

среднее                13674
высшее                  4702
СРЕДНЕЕ                  768
Среднее                  709
неоконченное высшее      667
ВЫСШЕЕ                   272
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64


Приведем данные к нижнему регистру, так как есть строки, отличающиеся регистром нескольких букв, для исключения неправильного название образования.

In [25]:
#Приведем значения в столбце 'education' к нижнему регистру
data['education'] = data['education'].str.lower()

In [26]:
#Проверяем наличие дубликатов после приведения к нижнему регистру
print(data.duplicated().sum())

17


In [27]:
data = data.drop_duplicates().reset_index(drop=True)
# удаление всех дубликатов из таблицы 

In [28]:
print(data.duplicated().sum())
#проверка

0


In [29]:
# Проверим результат
print(data['education'].value_counts())

среднее                15135
высшее                  5241
неоконченное высшее      743
начальное                282
ученая степень             6
Name: education, dtype: int64


Теперь данные об образование едины и не содержат неявных дубликатов

In [30]:
# Проверяем значения в столбце 'education_id'.
print(data['education_id'].value_counts())
#Данные корректны

1    15135
0     5241
2      743
3      282
4        6
Name: education_id, dtype: int64


In [31]:
# Проверяем значения в столбце 'family_status'.
print(data['family_status'].value_counts())
#Данные корректны

женат / замужем          12310
гражданский брак          4146
Не женат / не замужем     2805
в разводе                 1191
вдовец / вдова             955
Name: family_status, dtype: int64


In [32]:
# Проверяем значения в столбце 'family_status_id'.
print(data['family_status_id'].value_counts())
#Данные корректны

0    12310
1     4146
4     2805
3     1191
2      955
Name: family_status_id, dtype: int64


In [33]:
# Проверяем значения в столбце 'gender'.
print(data['gender'].value_counts())

F      14139
M       7267
XNA        1
Name: gender, dtype: int64


Присутствует одно значение XNA, неотносящееся к полу человека. Удалим строку со значением XNA

In [34]:
# Удалим строку со значением XNA
data = data[data['gender'] != 'XNA']
print(data['gender'].value_counts()) # проверка

F    14139
M     7267
Name: gender, dtype: int64


In [35]:
# Проверяем значения в столбце 'income_type'.
print(data['income_type'].value_counts())
#Данные корректны,дубликатов нет

сотрудник          11058
компаньон           5068
пенсионер           3821
госслужащий         1453
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64


In [36]:
# Проверяем значения в столбце 'debt'.
print(data['debt'].value_counts())
#Данные корректны

0    19666
1     1740
Name: debt, dtype: int64


In [37]:
# Проверяем значения в столбце 'total_income'.
print(data['total_income'].value_counts())

142594    1069
172357     501
118514     386
150447     145
162962       3
          ... 
390148       1
113661       1
111612       1
199675       1
264193       1
Name: total_income, Length: 18566, dtype: int64


In [38]:
# проверяем, что нет отрицательных значений в столбце 'total_income'.
print(data[data['total_income'] <= 0]) 
#Данные корректны

Empty DataFrame
Columns: [children, days_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose, age_group]
Index: []


In [39]:
# Проверим значения в столбце 'purpose'
print(data['purpose'].value_counts())

свадьба                                   791
на проведение свадьбы                     767
сыграть свадьбу                           764
операции с недвижимостью                  673
покупка коммерческой недвижимости         661
операции с жильем                         651
покупка жилья для сдачи                   650
операции с коммерческой недвижимостью     649
жилье                                     645
покупка жилья                             643
покупка жилья для семьи                   638
строительство собственной недвижимости    634
недвижимость                              632
операции со своей недвижимостью           624
строительство жилой недвижимости          621
покупка своего жилья                      619
строительство недвижимости                619
покупка недвижимости                      617
ремонт жилью                              606
покупка жилой недвижимости                604
на покупку своего автомобиля              504
заняться высшим образованием      

Во многих строках сама цель кредита одна, но записана по разному. Необходимо сгруппировать данные столбца 'purpose' по  целям кредита.

### Вывод
На этапе предобработки в данных обнаружились не только пропуски, но и всяческие виды дубликатов. Их удаление позволит провести анализ точнее. Проcмотрели уникальные значения в каждом столбце. Для удаления неявных дубликатов из столбца 'education', сначала привели строки значений к нижнему регистру. Чтобы избавиться от неявных дубликатов в столбце 'purpose' приведение к нижнему регистру недостаточно, так как цель кредита клиент может написать по своему усмотрению. Но среди разнообразия целей выдачи кредита, можно выделить и основные направления. Создадим категории для целей кредита. Для этого сначала сделаем лемматизацию столбца 'purpose'.


### Лемматизация

Для поиска целей кредита, записанных в разных формах, создадим новый столбец 'purpose_lem',содержащий в себе отлемматизированный столбец 'purpose'.

In [40]:
# импортируем pymystem3:
from pymystem3 import Mystem
m = Mystem()
#создадим новый столбец purpose_lem, в котором лемматизированы все слова в столбце purpose
data['purpose_lem'] =data['purpose'].apply(m.lemmatize)
data[['purpose','purpose_lem']].head(10) #вывод первоначальной и отлеммитизироанной цели кредита

,purpose,purpose_lem
0,покупка жилья,"[покупка, , жилье, \n]"
1,приобретение автомобиля,"[приобретение, , автомобиль, \n]"
2,покупка жилья,"[покупка, , жилье, \n]"
3,дополнительное образование,"[дополнительный, , образование, \n]"
4,сыграть свадьбу,"[сыграть, , свадьба, \n]"
5,покупка жилья,"[покупка, , жилье, \n]"
6,операции с жильем,"[операция, , с, , жилье, \n]"
7,образование,"[образование, \n]"
8,на проведение свадьбы,"[на, , проведение, , свадьба, \n]"
9,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]"


Подсчитаем количество уникальных лемматизированных слов. 

In [41]:
#Подсчитаем количество уникальных лемматизированных слов. 
# Вызовем специальный контейнер Counter из модуля collections.

from collections import Counter

text = ' '.join(data['purpose'])
lemmas = m.lemmatize(text)
dict_text = Counter(lemmas)
dict_text

Counter({'покупка': 5883,
         ' ': 54906,
         'жилье': 4452,
         'приобретение': 459,
         'автомобиль': 4295,
         'дополнительный': 903,
         'образование': 4003,
         'сыграть': 764,
         'свадьба': 2322,
         'операция': 2597,
         'с': 2913,
         'на': 2218,
         'проведение': 767,
         'для': 1288,
         'семья': 638,
         'недвижимость': 6334,
         'коммерческий': 1310,
         'жилой': 1225,
         'строительство': 1874,
         'собственный': 634,
         'подержать': 851,
         'свой': 2223,
         'со': 624,
         'заниматься': 904,
         'сделка': 940,
         'получение': 1312,
         'высокий': 1373,
         'подержанный': 111,
         'профильный': 434,
         'сдача': 650,
         'ремонт': 606,
         '\n': 1})

### Вывод
Cоздадли новый столбец 'purpose_lem',содержащий в себе отлемматизированный столбец 'purpose' и нашли уникальные значения. Теперь эти данные необходимо систематизировать.

### Категоризация данных

Вручную создадим словарь самых распространенных целей кредита. Создадим функцию, которая определяет цель кредита и выводит значение из созданного словаря, если значения в словаре нет, то выводит значение "иное".Создадим новый столбец с целями кредита по категориям 'purpose_new'.

Создадим словарь самых распространенных целей кредита.

In [42]:
# Создание словаря-синонимов

dict_purpose = {'недвижимость':['недвижимость','жилье','квартира'],
                 'автомобиль':['автомобиль','машина'],
                 'образование':['образование','учеба'],
                 'свадьба':['свадьба']}

Для внесения изменений в названия целей кредита воспользуемся соданным словарем названий: ключ словаря - окончательное название, значение - различные виды записи названий целей.

In [43]:
# Создание функции purpose_new

def purpose_new(purpose_lem):
    for key,value in dict_purpose.items():
        for purpose in purpose_lem:
            if purpose in value:
                return key
    else: return 'иное'
    
# Объявляется функция purpose_new с одним параметром: purpose_lem.
# В циклах проверяем вхождение каждого слова из столбца purpose_lem в каждое значение из словаря dict_purpose. 
# Функция возвращает ключ словаря dict_purpose(если есть вхождение в словарь dict_purpose), 
# либо значение "иное" (если ни одного слова нет в словаре)

data['purpose_new'] = data['purpose_lem'].apply(purpose_new)
# Cоздадим отдельный столбец с целями кредита по категориям. В его ячейках запишем значения, возвращаемые функцией.
# Для этого вызовем метод apply() он берёт значения столбца датафрейма и применяет к ним функцию из своего аргумента. 
# В нашем случае метод apply() следует вызвать для столбца 'purpose_lem', 
# так как в нём содержатся данные, которые функция примет на вход. Аргументом метода станет сама функция purpose_new.

data[['purpose','purpose_lem','purpose_new']].head(10)

,purpose,purpose_lem,purpose_new
0,покупка жилья,"[покупка, , жилье, \n]",недвижимость
1,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль
2,покупка жилья,"[покупка, , жилье, \n]",недвижимость
3,дополнительное образование,"[дополнительный, , образование, \n]",образование
4,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба
5,покупка жилья,"[покупка, , жилье, \n]",недвижимость
6,операции с жильем,"[операция, , с, , жилье, \n]",недвижимость
7,образование,"[образование, \n]",образование
8,на проведение свадьбы,"[на, , проведение, , свадьба, \n]",свадьба
9,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]",недвижимость


Проверка, что во всех строках прописана новая цель кредита из словаря самых распространенных целей кредита. Применим метод value_counts() для подсчёта значений каждой категории.

In [44]:
print(data['purpose_new'].value_counts()) #вывод значений,встречающихся в  'purpose_new'
print(data['purpose_new'].isnull().sum()) #проверка пропусков

недвижимость    10786
автомобиль       4295
образование      4003
свадьба          2322
Name: purpose_new, dtype: int64
0


Чтобы разделить заемщиков на группы по доходам Разделим их на категории с низким уровнем дохода, средним и высоким. Описательная статистика (среднее, стандартное отклонение, количество наблюдений, минимальное, максимальное и квартили) числовых столбцов может быть рассчитана с использованием .describe().

In [45]:
# Чтобы найти граничные значения уровня дохода (25%,75%) воспользуемся методом describe()
debt_total_income = data['total_income'].describe()
print(debt_total_income) #вывод результата

count    2.140600e+04
mean     1.653442e+05
std      9.824556e+04
min      2.066700e+04
25%      1.075505e+05
50%      1.425940e+05
75%      1.958600e+05
max      2.265604e+06
Name: total_income, dtype: float64


In [46]:
# На основании полученных расчетов, получаем
min_level = data['total_income'].describe().loc['25%']
max_level = data['total_income'].describe().loc['75%']
# создали переменные, в которых хранятся граничные значения суммы дохода по категориям

# Создание функции level_total_income
def level_total_income (total_income):
    if total_income <= min_level:
        return 'низкий доход'
    elif total_income >= max_level:
        return 'высокий доход'
    else:
        return 'средний доход'

# Объявляется функция level_total_income с одним параметром: total_income.
# Функция возвращает уровень дохода по значению суммы дохода 'total_income', используя правила
# - 'низкий доход' при значении total_income <= min_level
# - 'высокий доход' при значении total_income >= max_level
# - 'средний доход' во всех остальных случаях.

data['total_income_level'] = data['total_income'].apply(level_total_income)    
# Создадим отдельный столбец 'total_income_level' с уровнем дохода по категориям.
# В его ячейках запишем значения, возвращаемые функцией.
# Вызовим метод apply() для столбца 'total_income', так как в нём содержатся данные, которые функция примет на вход. 
# Аргументом метода станет сама функция level_total_income.

data[['total_income','total_income_level']].head(10)

,total_income,total_income_level
0,253875,высокий доход
1,112080,средний доход
2,145885,средний доход
3,267628,высокий доход
4,158616,средний доход
5,255763,высокий доход
6,240525,высокий доход
7,135823,средний доход
8,95856,низкий доход
9,144425,средний доход


Получаем общую информацию о данных. Убеждаемся, все выполнено успешно.

In [47]:
# получение общей информации о данных таблицы data
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21406 entries, 0 to 21406
Data columns (total 16 columns):
children              21406 non-null int64
days_employed         21406 non-null int64
dob_years             21406 non-null int64
education             21406 non-null object
education_id          21406 non-null int64
family_status         21406 non-null object
family_status_id      21406 non-null int64
gender                21406 non-null object
income_type           21406 non-null object
debt                  21406 non-null int64
total_income          21406 non-null int64
purpose               21406 non-null object
age_group             21406 non-null object
purpose_lem           21406 non-null object
purpose_new           21406 non-null object
total_income_level    21406 non-null object
dtypes: int64(7), object(9)
memory usage: 2.8+ MB


In [48]:
# вывод первых 10ти строк таблицы
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,purpose_lem,purpose_new,total_income_level
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,взрослые,"[покупка, , жилье, \n]",недвижимость,высокий доход
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,взрослые,"[приобретение, , автомобиль, \n]",автомобиль,средний доход
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,взрослые,"[покупка, , жилье, \n]",недвижимость,средний доход
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,взрослые,"[дополнительный, , образование, \n]",образование,высокий доход
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,взрослые,"[сыграть, , свадьба, \n]",свадьба,средний доход
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,молодежь,"[покупка, , жилье, \n]",недвижимость,высокий доход
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,взрослые,"[операция, , с, , жилье, \n]",недвижимость,высокий доход
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,взрослые,"[образование, \n]",образование,средний доход
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,взрослые,"[на, , проведение, , свадьба, \n]",свадьба,низкий доход
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,взрослые,"[покупка, , жилье, , для, , семья, \n]",недвижимость,средний доход


### Вывод
Для категоризации целей кредита выбран следующий словарь ['недвижимость','автомобиль','образование','свадьба']. Это основные цели получения кредита (максимальное количество заемщиков). Для категоризации дохода создали словарь ['средний доход' 'низкий доход' 'высокий доход']. Все необходимые для анализа данные подготовлены.


### Шаг 3. Исследование зависимостей

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Представим данные наглядно.
Первый способ:

In [49]:
debt_children = (data.groupby('children')
                 .agg({'family_status':'count','debt':'mean'})
                 .sort_values(by='debt',ascending=False)
                 .rename(columns={'family_status':'Заемщики','debt':'% должников'})
                )

#одной строкой: 
#группировка данных по столбцу 'children'
# в этой группировке методом 'mean' подсчёт среднего арифметического значений столбца 'debt'
# сортировка в порядке убывания по столбцу 'debt'
# переименование столбцов 'family_status' и 'debt'
# сохранение в debt_children
debt_children['% должников'] = debt_children['% должников'].round(4)*100 # переыодим в % и округляем
debt_children

,Заемщики,% должников
children,,
20,76,10.53
4,41,9.76
2,2052,9.45
1,4808,9.23
3,330,8.18
0,14090,7.54
5,9,0.00


### Вывод
Клиенты, у которых в семье 5 дтей всегда погашают кредит в срок. Из оставшихся категорий у бездетных клиентов самый низкий процент непогашения кредита (7.54%), самый высокий процент непогашения (10,5%) у семей с 20ю детьми. Значит, факт наличия детей у клиента влияет на факт погашения кредита в срок.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [52]:
data_pivot_family_status = data.pivot_table(index='family_status', aggfunc={'debt':'mean'}).rename(columns={'debt':'% должников'})
data_pivot_family_status['% должников'] = data_pivot_family_status['% должников'].round(4)*100
data_pivot_family_status

,% должников
family_status,
Не женат / не замужем,9.77
в разводе,7.14
вдовец / вдова,6.60
гражданский брак,9.36
женат / замужем,7.55


### Вывод
Результаты показывают, что процент непогашения кредита в срок больше у не женатых/не замужних и людей, состоящих в гражданском браке (9,77% и 9,36% соответственно). Вдовы/вдовцы самые ответственные клиенты, процент непогашения в срок 6,6%. Значит семейное положение клиента влияет на факт погашения кредита в срок. У клиентов в разводе или женатых/замужних процент примерно одинаков - 7,14% и 7,55% соответственно.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [53]:
data_pivot_total_income_level = data.pivot_table(index='total_income_level', aggfunc={'debt':'mean'}).rename(columns={'debt':'% должников'})
data_pivot_total_income_level['% должников'] = data_pivot_total_income_level['% должников'].round(4)*100
data_pivot_total_income_level

,% должников
total_income_level,
высокий доход,7.16
низкий доход,7.98
средний доход,8.69


### Вывод
Результаты показывают, что чаще непогашают кредит в срок клиенты со средним уровнем дохода (8,69%). Самыми добросовестными оказались клиенты с высоким уровнеи дохода (7,16%). У клиентов с низким уровнем дохода процент непогашения составил 7,98%. Значит, уровень дохода клиента влияет на факт погашения кредита в срок.

- Как разные цели кредита влияют на его возврат в срок?

In [54]:
data_pivot_purpose_new = data.pivot_table(index='purpose_new', aggfunc={'debt':'mean'}).rename(columns={'debt':'% должников'})
data_pivot_purpose_new['% должников'] = data_pivot_purpose_new['% должников'].round(4)*100
print(data_pivot_purpose_new)

              % должников
purpose_new              
автомобиль           9.36
недвижимость         7.25
образование          9.24
свадьба              8.01


### Вывод
Результаты показывают, что процент непогашения кредита в срок равен примерно 7-9% по каждой категории. По кредитам на недвижимость самый маленький процент непогашения (7,25), видимо потому, что кредит на недвижимость берется более осознанно и на длительный срок. Самый большой процент непогашения кредитов у клиентов, берущих кредиты на автомобиль (9,36%) и образование (9,24%). Процент должников, берущих кредит на свадьбу, составляет 8,01%.

### Шаг 4. Общий вывод

Исходный файл прочитан и превращен в структуру данных. Просмотрены данные о таблице. Проведена предподготовка данных:устранены пропуски и повторы. Данные были сгруппированы по целям кредита и уровню дохода клиентов. На основании проведенного анализа можно сделать вывод, что на факт погашения кредита в срок влияют: количество детей клиента, его семейное положение, уровень дохода, цель кредита. Дополнительно можно провести анализ платежеспособности клинта по его уровню образования, возрасту, полу, типу занятости.

In [55]:
# Например, анализ платежеспособности клинта по его уровню образования
data_pivot_education = data.pivot_table(index='education', aggfunc={'debt':'mean'}).rename(columns={'debt':'% должников'})
data_pivot_education['% должников'] = data_pivot_education['% должников'].round(4)*100
data_pivot_education

,% должников
education,
высшее,5.30
начальное,10.99
неоконченное высшее,9.16
среднее,9.01
ученая степень,0.00


Результаты показывают, что процент непогашения кредита в срок больше у людей, не имеющих высшее образование. Самые плохие показатели у людей с начальным образованием. Клиенты с ученой степенью всегда погашают кредиты в срок. Значит, уровень образования влияет на факт погашения кредита в срок. Чем больше человек учиться, тем он более надежный клиент.